In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/individual-test-spai-sorting-hat/submission.csv
/kaggle/input/individual-test-spai-sorting-hat/attendance.csv
/kaggle/input/individual-test-spai-sorting-hat/train.csv


In [ ]:
!pip install autogluon

In [3]:
train = pd.read_csv("/kaggle/input/individual-test-spai-sorting-hat/train.csv")
train.head()

,user_id,house
0,4b2569be-cd32-40aa-9cc4-9e13e55903de,house1
1,cdd098ee-4295-4839-b1a3-4d44840feacd,house3
2,226c662f-b1b3-40bc-9bfc-4f49a80b9041,house6
3,7889a31f-7351-4b4d-b872-e04e5e35b9dd,house2
4,0492584f-5dda-4ad6-ab2f-79c8d43a8753,house1


In [4]:
att = pd.read_csv("/kaggle/input/individual-test-spai-sorting-hat/attendance.csv")
att.head()

,user_id,datetime
0,5931fa6a-af6b-43b8-babb-28349854d406,2024-04-19 17:30:08.827555
1,5f1a2ecf-943d-446b-b94d-d7d1ad64213d,2024-04-19 17:30:11.405020
2,6e009a96-2b8b-4048-8b0c-20a7924a4b4f,2024-04-19 17:30:14.433098
3,cdd098ee-4295-4839-b1a3-4d44840feacd,2024-04-19 17:30:17.214453
4,f922a5d4-3651-46f7-aa98-307c045dd4b5,2024-04-19 17:30:21.556805


In [5]:
att['user_id'].value_counts()

user_id
797d702f-b117-4b78-9623-a52e043e30e7    68
3bd6dcac-904c-4614-be50-9c3949cb3405    68
81fd94c1-4f86-4441-a38e-7b770ce28aae    68
4af43656-b123-4a72-892b-7993a6355ed3    68
3a90e6b3-bc9f-49a7-997f-e954943de83e    68
                                        ..
71d0528a-188d-4aba-b7de-79ce11aba729    46
84aea5bb-fc9f-463e-8ed9-5c36ccb0dc7f    43
979ad268-eca3-4d3f-854e-d244f73b1748    43
c50ac909-8d9c-4713-b3d2-6add8d3e4acc    43
fed3dced-4009-461e-9411-a440c779e32c    37
Name: count, Length: 170, dtype: int64

## Merging Files

In [6]:
train['user_id'].dtype , att['user_id'].dtype

(dtype('O'), dtype('O'))

In [7]:
result = train.merge(att, on='user_id', how='outer')

In [8]:
result['user_id'].value_counts()

user_id
b10aa190-3d2a-4137-b674-8cb64824d72c    68
2ad18754-2522-4fa9-af12-bccdf42d3905    68
954ee6ef-741a-4199-882d-35b961f4453a    68
b542ca89-e19f-4f62-a46d-8034f956b019    68
bf565a52-d853-4fdb-a36d-2b4420313ddd    68
                                        ..
71d0528a-188d-4aba-b7de-79ce11aba729    46
84aea5bb-fc9f-463e-8ed9-5c36ccb0dc7f    43
979ad268-eca3-4d3f-854e-d244f73b1748    43
c50ac909-8d9c-4713-b3d2-6add8d3e4acc    43
fed3dced-4009-461e-9411-a440c779e32c    37
Name: count, Length: 170, dtype: int64

In [9]:
result['house'].isnull().sum()

4319

* รายชื่อคนที่หายไป

In [10]:
user_missing_house = result[result['house'].isnull()]['user_id'].unique()
user_missing_house[:5]

array(['0cb94071-a8ec-4885-8316-e0e4fbebec32',
       '0d9ff778-c1fe-4966-baa9-e35783893f39',
       '12584309-4699-485d-9f5b-a57ebfc4283a',
       '1a024edc-e397-4508-958e-7051ae278650',
       '1bedb648-5a50-4680-a09b-27240759f9a0'], dtype=object)

In [11]:
len(user_missing_house)

68

* เช็คจำนวนคนหายก่อน ว่าเท่ากับ Submission รึเปล่า

In [12]:
sub_df = pd.read_csv("/kaggle/input/individual-test-spai-sorting-hat/submission.csv")
sub_df.head()

,user_id,house
0,6e009a96-2b8b-4048-8b0c-20a7924a4b4f,house3
1,81c9f456-b5a2-4e28-ad69-eb7d89374d37,house1
2,dda733a2-2add-44a5-b888-81943923c2c6,house4
3,12584309-4699-485d-9f5b-a57ebfc4283a,NaN
4,c968fef9-5411-49d5-b6e1-d9a52add4ab8,NaN


In [13]:
sub_df['user_id']

0     6e009a96-2b8b-4048-8b0c-20a7924a4b4f
1     81c9f456-b5a2-4e28-ad69-eb7d89374d37
2     dda733a2-2add-44a5-b888-81943923c2c6
3     12584309-4699-485d-9f5b-a57ebfc4283a
4     c968fef9-5411-49d5-b6e1-d9a52add4ab8
                      ...                 
63    fed3dced-4009-461e-9411-a440c779e32c
64    60fea3c0-9066-49ea-beb2-b055ab2b4ff9
65    1bedb648-5a50-4680-a09b-27240759f9a0
66    ef836255-9ff1-48bd-9ee1-b3e205f96367
67    7474c4b5-d72d-4213-ad48-c8376a66cf35
Name: user_id, Length: 68, dtype: object

## เริ่มทำ Clustering

In [14]:
len(result)

10803

In [44]:
train = result[result['house'].notnull()]
test = result[result['house'].isnull()]
len(train), len(test)

(6484, 4319)

In [45]:
len(train)

6484

21 87 148

In [46]:
train['datetime'] = train['datetime'].astype('datetime64[ns]')
test['datetime'] = test['datetime'].astype('datetime64[ns]')

SettingWithCopyError: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

,user_id,house,datetime
129,0cb94071-a8ec-4885-8316-e0e4fbebec32,NaN,2024-04-19 17:47:00.347588
130,0cb94071-a8ec-4885-8316-e0e4fbebec32,NaN,2024-04-20 08:39:45.953953
131,0cb94071-a8ec-4885-8316-e0e4fbebec32,NaN,2024-04-20 12:47:48.319631
132,0cb94071-a8ec-4885-8316-e0e4fbebec32,NaN,2024-04-20 17:37:19.175257
133,0cb94071-a8ec-4885-8316-e0e4fbebec32,NaN,2024-04-21 08:33:59.209171
...,...,...,...
10798,fed3dced-4009-461e-9411-a440c779e32c,NaN,2024-05-15 17:39:52.867784
10799,fed3dced-4009-461e-9411-a440c779e32c,NaN,2024-05-16 08:32:05.743186
10800,fed3dced-4009-461e-9411-a440c779e32c,NaN,2024-05-16 12:31:13.931607
10801,fed3dced-4009-461e-9411-a440c779e32c,NaN,2024-05-16 17:41:13.753088


In [18]:
# Feature engineering the datetime

import numpy as np

# Timestamp - Feature Engineering
def create_feature(df):
    df['day_of_month'] = df['datetime'].dt.day
    df['day_of_week'] = df['datetime'].dt.dayofweek # Monday=0, Sunday=6
    df['is_weekend'] = df['datetime'].dt.dayofweek.isin([5, 6]).astype(int) # 1 if weekend, 0 otherwise
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df["hour"] = df["datetime"].dt.hour
    df["minute"] = df["datetime"].dt.minute
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
    df.drop(columns=["datetime"], inplace=True)
    return df

train = create_feature(train)
test = create_feature(test)

/tmp/ipykernel_155/874789905.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_of_month'] = df['datetime'].dt.day
/tmp/ipykernel_155/874789905.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['day_of_week'] = df['datetime'].dt.dayofweek # Monday=0, Sunday=6
/tmp/ipykernel_155/874789905.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

In [19]:
train

,user_id,house,day_of_month,day_of_week,is_weekend,month,day,hour,minute,hour_sin,hour_cos
0,0492584f-5dda-4ad6-ab2f-79c8d43a8753,house1,19,4,0,4,19,17,33,-9.659258e-01,-0.258819
1,0492584f-5dda-4ad6-ab2f-79c8d43a8753,house1,20,5,1,4,20,8,44,8.660254e-01,-0.500000
2,0492584f-5dda-4ad6-ab2f-79c8d43a8753,house1,20,5,1,4,20,12,43,1.224647e-16,-1.000000
3,0492584f-5dda-4ad6-ab2f-79c8d43a8753,house1,20,5,1,4,20,17,30,-9.659258e-01,-0.258819
4,0492584f-5dda-4ad6-ab2f-79c8d43a8753,house1,21,6,1,4,21,8,40,8.660254e-01,-0.500000
...,...,...,...,...,...,...,...,...,...,...,...
10761,fac56dec-f876-400b-921f-94012ed87d73,house6,15,2,0,5,15,17,58,-9.659258e-01,-0.258819
10762,fac56dec-f876-400b-921f-94012ed87d73,house6,16,3,0,5,16,8,53,8.660254e-01,-0.500000
10763,fac56dec-f876-400b-921f-94012ed87d73,house6,16,3,0,5,16,12,33,1.224647e-16,-1.000000
10764,fac56dec-f876-400b-921f-94012ed87d73,house6,16,3,0,5,16,17,36,-9.659258e-01,-0.258819


In [20]:
# Categorical - LabelEncoder

from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

cat_cols = ["user_id", "house"]

train[cat_cols] = train[cat_cols].apply(LabelEncoder().fit_transform)
# test[cat_cols] = test[test_cat_cols].apply(LabelEncoder().fit_transform)

# test_df.drop(columns="id", inplace=True)

/tmp/ipykernel_155/2841155416.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[cat_cols] = train[cat_cols].apply(LabelEncoder().fit_transform)


In [21]:
train.corr()

,user_id,house,day_of_month,day_of_week,is_weekend,month,day,hour,minute,hour_sin,hour_cos
user_id,1.000000,0.063394,0.004709,-0.001510,-0.002043,-0.007466,0.004709,-0.004915,0.018289,0.003778,-0.003408
house,0.063394,1.000000,0.007710,0.007127,0.001996,-0.011629,0.007710,0.000383,0.001716,-0.002092,-0.004605
day_of_month,0.004709,0.007710,1.000000,-0.009912,0.127178,-0.859389,1.000000,-0.071330,-0.023062,0.010991,0.170099
day_of_week,-0.001510,0.007127,-0.009912,1.000000,0.672662,-0.113366,-0.009912,-0.115112,-0.124062,0.027837,0.039112
is_weekend,-0.002043,0.001996,0.127178,0.672662,1.000000,-0.205406,0.127178,-0.238753,-0.092185,0.127621,0.092380
month,-0.007466,-0.011629,-0.859389,-0.113366,-0.205406,1.000000,-0.859389,0.054899,0.026534,-0.007160,-0.181940
day,0.004709,0.007710,1.000000,-0.009912,0.127178,-0.859389,1.000000,-0.071330,-0.023062,0.010991,0.170099
hour,-0.004915,0.000383,-0.071330,-0.115112,-0.238753,0.054899,-0.071330,1.000000,0.111108,-0.911719,0.080976
minute,0.018289,0.001716,-0.023062,-0.124062,-0.092185,0.026534,-0.023062,0.111108,1.000000,0.030029,-0.071042
hour_sin,0.003778,-0.002092,0.010991,0.027837,0.127621,-0.007160,0.010991,-0.911719,0.030029,1.000000,-0.252770


* ฉะนั้นใช้แต่วันที่ดีกว่า - K-means ไรงี้

In [22]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# # Normalize the features
# scaler = StandardScaler()
# X = train_cp.drop(columns="house")
# y = train_cp["house"]
# X_scaled = scaler.fit_transform(X)

# # Split into train and test
# X_train, X_test, y_train, y_test = train_test_split(
#     X_scaled, y, test_size=0.3, random_state=42, stratify=y
# )

In [23]:
train = train.drop(columns="user_id")

In [26]:
from autogluon.tabular import TabularPredictor
import pandas as pd

# Specify the column to predict (target variable)
label_column = "house"

# Initialize the AutoGluon TabularPredictor
predictor = TabularPredictor(label=label_column, eval_metric="f1_weighted").fit(train, time_limit=600, presets="best", ag_args_fit={'num_gpus': 1})

No path specified. Models will be saved in: "AutogluonModels/ag-20250705_105158"
Preset alias specified: 'best' maps to 'best_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.45 GB / 31.35 GB (93.9%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_st

In [38]:
len(test)

4319

In [43]:
test

,user_id,house,day_of_month,day_of_week,is_weekend,month,day,hour,minute,hour_sin,hour_cos
129,0cb94071-a8ec-4885-8316-e0e4fbebec32,5,19,4,0,4,19,17,47,-9.659258e-01,-0.258819
130,0cb94071-a8ec-4885-8316-e0e4fbebec32,4,20,5,1,4,20,8,39,8.660254e-01,-0.500000
131,0cb94071-a8ec-4885-8316-e0e4fbebec32,2,20,5,1,4,20,12,47,1.224647e-16,-1.000000
132,0cb94071-a8ec-4885-8316-e0e4fbebec32,2,20,5,1,4,20,17,37,-9.659258e-01,-0.258819
133,0cb94071-a8ec-4885-8316-e0e4fbebec32,0,21,6,1,4,21,8,33,8.660254e-01,-0.500000
...,...,...,...,...,...,...,...,...,...,...,...
10798,fed3dced-4009-461e-9411-a440c779e32c,1,15,2,0,5,15,17,39,-9.659258e-01,-0.258819
10799,fed3dced-4009-461e-9411-a440c779e32c,4,16,3,0,5,16,8,32,8.660254e-01,-0.500000
10800,fed3dced-4009-461e-9411-a440c779e32c,4,16,3,0,5,16,12,31,1.224647e-16,-1.000000
10801,fed3dced-4009-461e-9411-a440c779e32c,4,16,3,0,5,16,17,41,-9.659258e-01,-0.258819


In [28]:
test_cp = test.drop(columns=["user_id", "house"])

In [39]:
predictions = predictor.predict(test_cp)
len(predictions)

4319

In [42]:
test

,user_id,house,day_of_month,day_of_week,is_weekend,month,day,hour,minute,hour_sin,hour_cos
129,0cb94071-a8ec-4885-8316-e0e4fbebec32,5,19,4,0,4,19,17,47,-9.659258e-01,-0.258819
130,0cb94071-a8ec-4885-8316-e0e4fbebec32,4,20,5,1,4,20,8,39,8.660254e-01,-0.500000
131,0cb94071-a8ec-4885-8316-e0e4fbebec32,2,20,5,1,4,20,12,47,1.224647e-16,-1.000000
132,0cb94071-a8ec-4885-8316-e0e4fbebec32,2,20,5,1,4,20,17,37,-9.659258e-01,-0.258819
133,0cb94071-a8ec-4885-8316-e0e4fbebec32,0,21,6,1,4,21,8,33,8.660254e-01,-0.500000
...,...,...,...,...,...,...,...,...,...,...,...
10798,fed3dced-4009-461e-9411-a440c779e32c,1,15,2,0,5,15,17,39,-9.659258e-01,-0.258819
10799,fed3dced-4009-461e-9411-a440c779e32c,4,16,3,0,5,16,8,32,8.660254e-01,-0.500000
10800,fed3dced-4009-461e-9411-a440c779e32c,4,16,3,0,5,16,12,31,1.224647e-16,-1.000000
10801,fed3dced-4009-461e-9411-a440c779e32c,4,16,3,0,5,16,17,41,-9.659258e-01,-0.258819


In [32]:
house_mapping = {
    0: "house1",
    1: "house2",
    2: "house3",
    3: "house4",
    4: "house5",
    5: "house6"
}

In [30]:
len(predictions)

4319

In [33]:
sub_df = pd.read_csv("/kaggle/input/individual-test-spai-sorting-hat/submission.csv")
sub_df.head()

,user_id,house
0,6e009a96-2b8b-4048-8b0c-20a7924a4b4f,house3
1,81c9f456-b5a2-4e28-ad69-eb7d89374d37,house1
2,dda733a2-2add-44a5-b888-81943923c2c6,house4
3,12584309-4699-485d-9f5b-a57ebfc4283a,NaN
4,c968fef9-5411-49d5-b6e1-d9a52add4ab8,NaN


In [36]:
len(sub_df)

68

In [35]:
sub_df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,user_id,house
0,6e009a96-2b8b-4048-8b0c-20a7924a4b4f,NaN
1,81c9f456-b5a2-4e28-ad69-eb7d89374d37,NaN
2,dda733a2-2add-44a5-b888-81943923c2c6,NaN
3,12584309-4699-485d-9f5b-a57ebfc4283a,NaN
4,c968fef9-5411-49d5-b6e1-d9a52add4ab8,NaN
...,...,...
63,fed3dced-4009-461e-9411-a440c779e32c,NaN
64,60fea3c0-9066-49ea-beb2-b055ab2b4ff9,NaN
65,1bedb648-5a50-4680-a09b-27240759f9a0,NaN
66,ef836255-9ff1-48bd-9ee1-b3e205f96367,NaN


In [50]:
predictions.to_csv("predictions.csv")